In [33]:
import pandas as pd

In [71]:
from nomad.datamodel.metainfo.basesections import (
    PureSubstanceComponent,
    PureSubstanceSection,
)
from nomad_material_processing.vapor_deposition import (
    Pressure,
    Temperature,
    VolumetricFlowRate,
)
from nomad_material_processing.vapor_deposition.cvd import (
    BubblerEvaporator,
)

from pdi_nomad_plugin.utils import (
    clean_col_names,
    fill_quantity,
)

In [72]:
mainfile = '~/CT500/NOMAD/FILES/MOVPE_CNR_PARMA/test-files_giu24/101/101.xlsx'
xlsx = pd.ExcelFile(mainfile)
growthrun_sheet = pd.read_excel(xlsx, 'GrowthRun', comment='#')

In [97]:
def split_list_by_element(lst, element):
    """
    Split a list by an element
    """
    indices = [i for i, x in enumerate(lst) if element in x]
    start = 0
    result = []
    for index in indices:
        result.append(lst[start:index])
        start = index
    result.append(lst[start:])
    return result


def rename_block_cols(string, block_cols, initial_col):
    """
    Rename columns by splitting on '.' and appending the index of the block column
    """

    split_list = split_list_by_element(string, initial_col)

    new_columns = []
    for index, chunk in enumerate(split_list):
        bubbler = [f'{i}.{index}' for i in chunk if i in block_cols]
        other_cols = [i for i in chunk if i not in block_cols]
        if bubbler:
            new_columns.extend(bubbler)
            bubbler = []
        if other_cols:
            new_columns.extend(other_cols)
            other_cols = []
    return new_columns


growthrun_cols = clean_col_names(growthrun_sheet)
bubbler_cols = [
    'Bubbler Temp',
    'Bubbler Pressure',
    'Partial Pressure',
    'Bubbler Dilution',
    'Inject',
    'Bubbler Flow',
    'Bubbler Material',
    'Bubbler Valve',
]
bub_first_col = 'Bubbler Material'
growthrun_cols = rename_block_cols(growthrun_cols, bubbler_cols, bub_first_col)
gas_source_quantities = [
    'Gas Cylinder Material',
    'Dilution in Cylinder',
    'Flow from MFC',
    'Effective  Flow',
    'Gas Partial Pressure',
    'Cylinder Pressure',
    'Gas Valve',
]
gas_first_col = 'Gas Cylinder Material'
growthrun_cols = rename_block_cols(growthrun_cols, gas_source_quantities, gas_first_col)
growthrun_sheet.columns = growthrun_cols

In [98]:
def populate_bubblers(growth_step: pd.DataFrame, bub_first_col: str):
    bubblers = []

    columns = split_list_by_element(growth_step.index, bub_first_col)
    bubblers = []
    for column in columns:
        if any(bub_first_col in item for item in column):
            quantity, sep, index = column[0].rpartition('.')
            bubbler_obj = BubblerSourceIMEM()
            bubbler_obj.material = [PureSubstanceComponent]
            bubbler_obj.material[0].pure_substance = PureSubstanceSection()
            bubbler_obj.vapor_source = BubblerEvaporator()
            bubbler_obj.vapor_source.temperature = Temperature()
            bubbler_obj.vapor_source.pressure = Pressure()
            bubbler_obj.vapor_source.total_flow_rate = VolumetricFlowRate()
            my = f'Bubbler Material{sep}{index}'
            bubbler_obj.name = fill_quantity(
                growth_step, f'Bubbler Material{sep}{index}'
            )
            bubbler_obj.material[0].substance_name = fill_quantity(
                growth_step, f'Bubbler Material{sep}{index}'
            )
            bubbler_obj.material[0].pure_substance.name = fill_quantity(
                growth_step, f'Bubbler Material{sep}{index}'
            )
            bub_temp = fill_quantity(
                growth_step, f'Bubbler Temp{sep}{index}', read_unit='celsius'
            )
            if bub_temp:
                bubbler_obj.vapor_source.temperature.set_time = pd.Series([0])
                bubbler_obj.vapor_source.temperature.set_value = pd.Series(bub_temp)

            bub_press = fill_quantity(
                growth_step, f'Bubbler Pressure{sep}{index}', read_unit='mbar'
            )
            if bub_press:
                bubbler_obj.vapor_source.pressure.set_time = pd.Series([0])
                bubbler_obj.vapor_source.pressure.set_value = pd.Series([bub_press])

            bub_flow = fill_quantity(
                growth_step, f'Bubbler Flow{sep}{index}', read_unit='cm **3 / minute'
            )
            if bub_flow:
                bubbler_obj.vapor_source.total_flow_rate.set_time = pd.Series([0])
                bubbler_obj.vapor_source.total_flow_rate.set_value = pd.Series(
                    [bub_flow]
                )

            bub_dilution = fill_quantity(growth_step, f'Bubbler Dilution{sep}{index}')
            if bub_dilution:
                bubbler_obj.vapor_source.dilution = bub_dilution

            bubblers.append(bubbler_obj)
    return bubblers

In [101]:
for step_index, step in growthrun_sheet.iterrows():
    bubblers = populate_bubblers(step, 'Bubbler Material')
print(bubblers)

[BubblerSourceIMEM(material, vapor_source), BubblerSourceIMEM(material, vapor_source)]


In [42]:
for step_index, step in growthrun_sheet.iterrows():
    growth_step = step.to_frame().T
    print('YOOO')
    print(growth_step)
    print(step)

YOOO
             Name Step Duration Temperature Pressure Rotation Carrier Gas  \
0  undoped buffer  1.0      2.0       800.0    100.0    300.0       Argon   

  Flow Metal Carrier Metal Inner Valve Metal Outer Valve  ... Flow from MFC.1  \
0              700.0             100.0             100.0  ...             0.0   

  Effective  Flow.1 Gas Partial Pressure.1 Cylinder Pressure.1 Gas Valve.1  \
0               0.0                    0.0                 0.0         0.0   

  MIST Source 1 MIST Flow MFC MIST Valve                 Notes Unnamed: 50  
0           NaN           0.0        0.0  undoped buffer layer         NaN  

[1 rows x 51 columns]
Name                             undoped buffer
Step                                        1.0
Duration                                    2.0
Temperature                               800.0
Pressure                                  100.0
Rotation                                  300.0
Carrier Gas                               Argon
Flow Me